In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [6]:
df = pd.DataFrame(columns=['Faculty Member', 'Ph.D.', 'Masters', 'Bachelors'])

pgnum_list = np.arange(0,50,1)

for pgnum in pgnum_list:
    url = f'https://www.engr.washington.edu/facultyfinder?page={pgnum}'
    # Send a GET request to the faculty finder page
    response = requests.get(url)
    
    # Parse the HTML response with Beautiful Soup
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find_all('table')[-1]
    
    # Verifying tables and their classes
    # print('Classes of each table:')
    # for table in soup.find_all('table'):
    #     print(table.get('class'))
    
    
    # Loop through each row in the table and extract the faculty information
    for row in table.find_all('tr'):
        columns = row.find_all('td')
        
        # Check if the row contains faculty information
        if len(columns) != 0:
            # Extract the faculty member's name and personal page link
            name = columns[1].text.strip('\n').splitlines()[0]
            link = columns[1].find('a')['href']
            
            # Send a GET request to the faculty member's personal page
            try: response = requests.get(link)
            except: pass
            
            # Parse the HTML response with Beautiful Soup
            soup = BeautifulSoup(response.content, 'html.parser')
            
            # Find the education section of the faculty member's personal page
            education_section = soup.find('h2', text='Education')
            
            # If the education section exists, extract the education data
            if education_section is not None:
                education_section = education_section.find_next_sibling()
                
                # Split the education data by newline characters and remove empty strings
                education_data = [d.strip() for d in education_section.text.strip().split('\n') if d.strip()]
                
                # Loop through each education record and categorize it
                phd = ''
                masters = ''
                bachelors = ''
                for data in education_data:
                    if 'Ph.D.' in data or 'PhD' in data:
                        try:
                            phd += data.split(',')[1] + ', '
                        except IndexError:
                            phd += 'Issue'
                    elif 'M.S.' in data or 'Master' in data or 'MS' in data or 'MEng' in data or 'M.E.' in data:
                        try:
                            masters += data.split(',')[1] + ', '
                        except IndexError:
                            masters += 'Issue'
                    elif 'B.S.' in data or 'B.A.' in data:
                        try:
                            bachelors += data.split(',')[1] + ', '
                        except IndexError:
                            bachelors += 'Issue'
                
                # Remove trailing commas
                phd = phd.strip(', ')
                masters = masters.strip(', ')
                bachelors = bachelors.strip(', ')
            else:
                phd = 'Not available'
                masters = 'Not available'
                bachelors = 'Not available'
            
            # Add the faculty member's name and education data to the dataframe
            df = df.append({'Faculty Member': name, 'Ph.D.': phd, 'Masters': masters, 'Bachelors': bachelors}, ignore_index=True)

# Print the resulting dataframe
# print(df)

C:\Users\profe\AppData\Local\Temp\ipykernel_5868\3978838614.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Faculty Member': name, 'Ph.D.': phd, 'Masters': masters, 'Bachelors': bachelors}, ignore_index=True)
C:\Users\profe\AppData\Local\Temp\ipykernel_5868\3978838614.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Faculty Member': name, 'Ph.D.': phd, 'Masters': masters, 'Bachelors': bachelors}, ignore_index=True)
C:\Users\profe\AppData\Local\Temp\ipykernel_5868\3978838614.py:78: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Faculty Member': name, 'Ph.D.': phd, 'Masters': masters, 'Bachelors': bachelors}, ignore_index=True)
C:\Users\profe\AppData\Local\Temp\ipyker

In [7]:
df.to_csv(path_or_buf='./eng.csv')

In [8]:
uniq = df['Ph.D.'].describe().unique()
print(uniq) 

[395 106 'Not available' 188]


In [13]:
df['Ph.D.'].hist(xrot=90,bins=uniq[1],figsize=(18,9),backend='plotly')